# Increasing the Maximum Message Size for gRPC


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)

# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

An example start command using the kvm2 driver would look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

Install the custom resource definition

In [ ]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon --set ambassador.enabled=true

Check all services are running before proceeding.

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

In [ ]:
!pygmentize resources/model_long_timeouts.json

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [ ]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

In [ ]:
!kubectl rollout status deploy/long-to-test-7cd068f

## Get predictions - no grpx max message size

In [3]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="model-long-timeout",namespace="seldon", 
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)

Send a small request which should suceed.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Send a large request which will be above the default gRPC message size and will fail.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
print(r.success,r.msg)

In [ ]:
!kubectl delete -f resources/model_long_timeouts.json

In [ ]:
!helm delete seldon-core --purge

# Allowing larger gRPC messages

Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [ ]:
!pygmentize resources/model_grpc_size.json

In [ ]:
!kubectl create -f resources/model_grpc_size.json -n seldon

In [ ]:
!kubectl rollout status deploy/test-deployment-grpc-size-fd60a01 

Send a request via ambassador. This should succeed.

In [4]:
sc = SeldonClient(deployment_name="seldon-model",namespace="seldon",
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
print(r.success,r.msg)

True 


In [ ]:
!kubectl delete -f resources/model_grpc_size.json -n seldon